In [ ]:
# ensure basic colab requiremetns inplace
!pip install einops
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# connect google drive for full files and results transfer
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r /content/gdrive/MyDrive/full_test/ /content
%cd full_test
!ls

Mounted at /content/gdrive
/content/full_test
 data		   layers							     results.ipynb
 data_provider	  'MM DSM 500 - Final Project draft 2.docx'			     results..xlsx
 embed		  'MM DSM 500 - Final Project (working main) (AutoRecovered).docx'   run_test.py
 exp		   models							     scripts
 full_test.ipynb   requirements.txt						     utils
 full_test.py	   results


In [ ]:
# import requirements 
import os
from exp.exp_main import Exp_Main
import torch
import numpy as np
import time
from utils.set_args import set_args

# func to handle the full test in colab
def run_test_colab(model_name='Informer', data_name='ECL', single_pred=96, multi_test=True, multi_pred=[96, 192, 336, 720]):
    # start time
    full_start_time = time.time()

    root_path='./data/' # data root path
    random_seed=2024 # random seed for reproduciblity


    # get basic varibles for function
    model_name = model_name
    data_name = data_name
    data_path = data_name + '.csv'


    # get model num from model dict
    models = {'Informer' : 'Model-1',
              'PatchTST' : 'Model-2',
              'DLinear' : 'Model-3',
              'NLinear' : 'Model-4',
              'FEDformer' : 'Model-5',
              'Linear_PatchTST' : 'Model-6',
              'PatchFED' : 'Model-7',
              'RevIN_PatchTST' : 'Model-8',
              'RevIN_Linear' : 'Model-9'}

    model_num = models[model_name]

    # get model ID name
    if data_name == 'ECL':
        model_id_name='Electricity' # model id name
    elif data_name == 'PJM':
        model_id_name='PJM' # model id name

    # set target

    if data_name == 'ECL':
        target ='OT'
    elif data_name == "PJM":
        target ='FE'

    # prediction types and lengths
    multi_test = multi_test
    single_pred = single_pred
    multi_pred = multi_pred

    print(data_name, model_id_name)

    # create the exprimental class # from exp_main.py
    Exp = Exp_Main

    # copy results if exist in to drive as backup
    !cp -r /content/full_test/results /content/gdrive/MyDrive/full_test/

    # run the test check if multi or single test
    if multi_test:
        for seq_len in [96]: #192, 336, 720 other options for seq len
            for pred_len in multi_pred:

                seq_len = seq_len
                # start time
                start_pred_len = time.time()

                # set training parameters
                pred_len = pred_len

                # get args and setting define based on input variables 
                args, setting = set_args(model_num, data_name, seq_len, model_name, root_path, data_path, model_id_name, random_seed, pred_len, target, full_start_time)

                print(setting)
                # results path
                folder_path = os.path.join('./results/', setting)
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)
                test_file_name = folder_path + '/Model training and test.txt'
                test_f = open(test_file_name, 'a')


                # set experiments into the experiment class
                exp = Exp(args)

                # train
                print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)

                # test
                print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                exp.test(setting)


                torch.cuda.empty_cache()

                # pred run tim
                end_pred_len = time.time()
                elapsed_pred_len = end_pred_len - start_pred_len

                # show outputs of time 
                print("Length of train/test for pred length {} : {} sec or {} mins ", pred_len, elapsed_pred_len, elapsed_pred_len/60)
                test_f = open(test_file_name, 'a')
                test_f.write("\nTime elapsed (s): {} \n".format(pred_len))
                test_f.write("Time elapsed (s): {} \n".format(elapsed_pred_len/60))
                test_f.write("Time elapsed (s): {} \n".format((elapsed_pred_len/60)/60))

                # backup results 
                !cp -r /content/full_test/results /content/gdrive/MyDrive/full_test/
            # clear cache for multi runs 
            torch.cuda.empty_cache()

    else:
            seq_len = 96 # default value
            # set training parameters
            pred_len = single_pred

            # get args and setting define based on input variables 
            args, setting = set_args(model_num, data_name, seq_len, model_name, root_path, data_path, model_id_name, random_seed, pred_len, target, full_start_time)

            print(setting)
            # results path
            folder_path = os.path.join('./results/', setting)
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            test_file_name = folder_path + '/Model training and test.txt'
            test_f = open(test_file_name, 'a')
            test_f.write('\n')

            # set experiments
            exp = Exp(args)

            # train
            print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
            exp.train(setting)

            # test
            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.test(setting)

            torch.cuda.empty_cache()

    # full time
    full_end = time.time()
    full_elapsed = full_end - full_start_time
    # show outputs of time 
    print("Time elapsed (s): {} ".format(full_elapsed))
    print("Time elapsed (mins): {} ".format(full_elapsed/60))
    print("Time elapsed (hrs): {}".format((full_elapsed/60)/60))
    test_f = open(test_file_name, 'a')
    test_f.write("\nTime elapsed (s): {} \n".format(full_elapsed))
    test_f.write("Time elapsed (s): {} \n".format(full_elapsed/60))
    test_f.write("Time elapsed (s): {} \n".format((full_elapsed/60)/60))
    # back up results
    !cp -r /content/full_test/results /content/gdrive/MyDrive/full_test/

In [ ]:
# run the full test - input the model name, data name, multi test and preds 
run_test_colab(model_name='Linear_PatchTST', data_name='ECL', multi_test=True, multi_pred=[96, 192, 336, 720]) #

ECL Electricity
Linear_PatchTST_Model-6_ECL_ftM_sl720_ll48_pl96_dm512_nh8_el2_dl1_df2048_atprob_fc3_ebtimeF_dtTrue_mxNone_Exp_0
Linear_PatchTST_Model-6_ECL_ftM_sl720_ll48_pl96_dm512_nh8_el2_dl1_df2048_atprob_fc3_ebtimeF_dtTrue_mxNone_Exp_0
Use GPU: cuda:0
>>>>>>>start training : Linear_PatchTST_Model-6_ECL_ftM_sl720_ll48_pl96_dm512_nh8_el2_dl1_df2048_atprob_fc3_ebtimeF_dtTrue_mxNone_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 17597
val 2537
test 5165
	iters: 100, epoch: 1 | loss: 0.2021187
	speed: 0.1801s/iter; left time: 1959.6805s
	iters: 200, epoch: 1 | loss: 0.2147429
	speed: 0.3115s/iter; left time: 3358.1575s
	iters: 300, epoch: 1 | loss: 0.1734799
	speed: 0.4419s/iter; left time: 4719.7550s
	iters: 400, epoch: 1 | loss: 0.1676506
	speed: 0.5704s/iter; left time: 6035.2946s
	iters: 500, epoch: 1 | loss: 0.1548763
	speed: 0.6966s/iter; left time: 7300.9911s
Epoch: 1 cost time: 71.62498784065247
Epoch: 1, Steps: 549 | Train Loss: 0.2022204 Vali Loss: 0.1289941 Test Loss: 0.1513274
Valida


run_test(model_name='DLinear', single_pred=96)

--model_name	- * default DLinear options below

--data_name 	- optional default ECL, options below

--seq_len	- optional default 96

--target 	- optional default None- will give the one based on the dataset(last col in dataset)

--single_pred	- * default 96

--multi_test 	- True/ False

--multi_pred 	- optional default if True above [96, 192, 336, 720]

Model_names:
Informer 	: Model-1

PatchTST 	: Model-2

DLinear 	: Model-3

NLinear 	: Model-4

FEDformer 	: Model-5

Linear_PatchTST : Model-6

PatchFED 	: Model-7

RevIN_PatchTST 	: Model-8

RevIN_Linear 	: Model-9



Data_names:

ECL

PJM